In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation one our value function change is less than theta for all states.
        discount_factor: lambda discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [4]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: Lambda discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    # MAX_Y, MAX_X = env.shape
    # grid = np.arange(env.nS).reshape(env.shape)
    # while True:
    #    V = policy_eval_fn(policy, env, discount_factor)
    #    t = np.min(V)
    #    new_policy = np.zeros([env.nS, env.nA])
    #    it = np.nditer(grid, flags=['multi_index'])
    #    while not it.finished:
    #        s = it.iterindex
    #        y, x = it.multi_index
    #        ns = {}
    #        ns[0] = s if y == 0 else s - MAX_X  # UP
    #        ns[1] = s if x == (MAX_X - 1) else s + 1  # RIGHT
    #        ns[2] = s if y == (MAX_Y - 1) else s + MAX_X  # DOWN
    #        ns[3] = s if x == 0 else s - 1  # LEFT
    #        max_v = t
    #        for i in range(4):
    #            if V[ns[i]] > max_v:
    #                max_v = V[ns[i]]
    #                a = i
    #        new_policy[s][a] = 1.0
    #        it.iternext()
    #    if np.array_equal(new_policy, policy):
    #        break
    #    policy = new_policy
    #  上面的写法是错误的,policy的确定是根据action value,而不是只根据state value
    while True:
        V = policy_eval_fn(policy, env, discount_factor)
        flag = False  # 记录是否更新policy
        for s in range(env.nS):
            old_action = np.argmax(policy[s])
            action_value = np.zeros(env.nA)
            for a in range(env.nA):
                for prob, next_state, reward, done in env.P[s][a]:
                    # reward实际上可以不用放到这里,而是在action执行时就可以加上
                    # 但是因为这里实现的时候把reward的返回放在了抵达next_state的时候
                    # 所以只能通过把reward和prob相乘的方式来把reward加入到action_value中
                    action_value[a] += prob * (reward + discount_factor * V[next_state])
            new_action = np.argmax(action_value)
            if new_action != old_action:
                flag = True
            # 这里有一个bug,如果认为new_action和old_action相同时可以不用更新这个状态的policy
            # 就会出现错误. 这是因为初始的时候给的policy不是确定性的,所以会出现
            # old_action = np.argmax(policy[s])返回的结果并不是真正的old_action
            # 因此我们要确保policy更新成确定性的.
            policy[s] = np.eye(env.nA)[new_action]
        if not flag:
            break
    return policy, V

In [5]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [6]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)